In [115]:
using CSV, JuMP, Gurobi, DataFrames

In [117]:
GUROBI_ENV = Gurobi.Env();

Academic license - for non-commercial use only


In [234]:
PLAYERS_DATA_PATH = "../data/FanDuel-NBA-2019-11-26-40944-players-list.csv"

"../data/FanDuel-NBA-2019-11-26-40944-players-list.csv"

In [235]:
players = CSV.read(PLAYERS_DATA_PATH)

,Id,Position,First Name,Nickname,Last Name,FPPG
,String,String,String,String,String,Float64
1,40944-84669,PG,Luka,Luka Doncic,Doncic,57.5875
2,40944-55062,C,Nikola,Nikola Jokic,Jokic,40.3133
3,40944-14513,PF,Kawhi,Kawhi Leonard,Leonard,48.7667
4,40944-15595,SG,Bradley,Bradley Beal,Beal,45.8429
5,40944-12341,SF,Paul,Paul George,George,40.6
6,40944-63122,PF,Kristaps,Kristaps Porzingis,Porzingis,37.96
7,40944-66108,PG,Jamal,Jamal Murray,Murray,35.3133
8,40944-23776,C,Montrezl,Montrezl Harrell,Harrell,36.7412
9,40944-9662,SG,Lou,Lou Williams,Williams,34.5412


In [236]:
MAX_PG = 2
MAX_SG = 2
MAX_SF = 2
MAX_PF = 2
MAX_C = 1
BUDGET = 60000

60000

In [237]:
fantasy_points = players[:,:FPPG];
PG = Int.(players[:,:Position].=="PG")
SG = Int.(players[:,:Position].=="SG")
SF = Int.(players[:,:Position].=="SF");
PF = Int.(players[:,:Position].=="PF")
C  = Int.(players[:,:Position].=="C")
salary = players[:,:Salary];
injury = 1 .- ismissing.(players[:,Symbol("Injury Indicator")]) 
o_injury = Int.(Missings.coalesce.(players[:,Symbol("Injury Indicator")], 0).=="O")
q_injury = Int.(Missings.coalesce.(players[:,Symbol("Injury Indicator")], 0).=="Q")
p_injury = Int.(Missings.coalesce.(players[:,Symbol("Injury Indicator")], 0).=="P");

In [245]:
nb_players = size(players)[1]

lineup = Model(solver=GurobiSolver(OutputFlag=0, GUROBI_ENV))
@variable(lineup, z[i=1:nb_players], Bin)
@objective(lineup, Max, sum(fantasy_points.*z))
@constraint(lineup, sum(salary.*z) <= BUDGET)
@constraint(lineup, sum(PG.*z) == MAX_PG)
@constraint(lineup, sum(SG.*z) == MAX_SG)
@constraint(lineup, sum(SF.*z) == MAX_SF)
@constraint(lineup, sum(PF.*z) == MAX_PF)
@constraint(lineup, sum(C.*z) == MAX_C)
@constraint(lineup, z .<= (1 .-injury))

solve(lineup)
z_opt = Int.(getvalue(z))
names = sort(players[z_opt.==1,[:Nickname,:Position]])

,Nickname,Position
,String,String
1,Bradley Beal,SG
2,Delon Wright,SG
3,Ivica Zubac,C
4,Jalen Brunson,PG
5,Kawhi Leonard,PF
6,Luka Doncic,PG
7,Rui Hachimura,PF
8,Troy Brown,SF
9,Will Barton,SF


In [359]:
NB_LINEUPS = 10
MAX_OVERLAP = 6

nb_players = size(players)[1]
model = Model(solver=GurobiSolver(OutputFlag=0, GUROBI_ENV))
@variable(model, z[i=1:nb_players], Bin)
@objective(model, Max, sum(fantasy_points.*z))
@constraint(model, sum(salary.*z) <= BUDGET)
@constraint(model, sum(PG.*z) == MAX_PG)
@constraint(model, sum(SG.*z) == MAX_SG)
@constraint(model, sum(SF.*z) == MAX_SF)
@constraint(model, sum(PF.*z) == MAX_PF)
@constraint(model, sum(C.*z) == MAX_C)
@constraint(model, z .<= (1 .- injury))
solve(model)
x = Int.(getvalue(z))
lineups = players[x.==1,[:Nickname, :Position, :FPPG]]    
names!(lineups, Symbol.([string("Nickname_",1), string("Position_",1), string("FP_",1)]))    
@constraint(model, sum(x.*z) <= MAX_OVERLAP)
for i=2:NB_LINEUPS
    solve(model)
    x = hcat(x,Int.(getvalue(z)))
    lineups_names = players[x[:,i].==1, [:Nickname, :Position, :FPPG]]
    names!(lineups_names, Symbol.([string("Nickname_",i), string("Position_",i), string("FP_",i)]))    
    lineups = hcat(lineups, lineups_names)
    @constraint(model, sum(x[:,i].*z) <= MAX_OVERLAP)
end

In [360]:
lineups

,Nickname_1,Position_1,FP_1,Nickname_2,Position_2,FP_2,Nickname_3
,String,String,Float64,String,String,Float64,String
1,Luka Doncic,PG,57.5875,Luka Doncic,PG,57.5875,Luka Doncic
2,Kawhi Leonard,PF,48.7667,Kawhi Leonard,PF,48.7667,Kawhi Leonard
3,Bradley Beal,SG,45.8429,Montrezl Harrell,C,36.7412,Bradley Beal
4,Will Barton,SF,31.4231,Lou Williams,SG,34.5412,Will Barton
5,Rui Hachimura,PF,23.2,Will Barton,SF,31.4231,Moritz Wagner
6,Delon Wright,SG,21.0938,Delon Wright,SG,21.0938,Delon Wright
7,Ivica Zubac,C,21.5294,J.J. Barea,PG,17.45,Ish Smith
8,Troy Brown,SF,18.4091,JaMychal Green,PF,18.0941,JaMychal Green
9,Jalen Brunson,PG,16.3625,Troy Brown,SF,18.4091,Troy Brown


In [361]:
aggregate(lineups[:,Symbol.([string("FP_",i) for i=1:NB_LINEUPS])], sum)

,FP_1_sum,FP_2_sum,FP_3_sum,FP_4_sum,FP_5_sum,FP_6_sum,FP_7_sum,FP_8_sum
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,284.215,284.107,284.031,283.994,283.976,283.166,282.717,282.567
